In [2]:
import pandas as pd
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm_notebook as tqdm
from tqdm import trange
from sklearn.preprocessing import Normalizer

In [2]:
filepath = "NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
emolex_df = pd.read_csv(filepath,  names=["word", "emotion", "association"], sep='\t')
emolex_words = emolex_df.pivot(index='word',columns='emotion',values='association').reset_index()
emolex_words

emotion,word,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,NaN,0,0,0,0,0,0,0,0,0,0
1,aback,0,0,0,0,0,0,0,0,0,0
2,abacus,0,0,0,0,0,0,0,0,0,1
3,abandon,0,0,0,1,0,1,0,1,0,0
4,abandoned,1,0,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
14177,zone,0,0,0,0,0,0,0,0,0,0
14178,zoo,0,0,0,0,0,0,0,0,0,0
14179,zoological,0,0,0,0,0,0,0,0,0,0
14180,zoology,0,0,0,0,0,0,0,0,0,0


In [3]:
df_merge = pd.read_csv('full_merge.csv')
df_merge = df_merge.dropna()
df_merge.head()
print(df_merge.shape)

(337337, 5)


In [72]:
# drugs_grouped = df_merge.groupby('Drug').count()
# drug_review_grouping = drugs_grouped.merge(df_merge,on='Drug',how='inner')
# df_new = drug_review_grouping.drop_duplicates()
# df_new.shape
#df_new.to_csv('df_new.csv',index=False)

(337337, 9)

In [4]:
#df_drug_reviews_merged = df_new[['Drug','Reviews_y']].rename(columns={'Reviews_y':'Reviews'})
df_merge['Reviews'] = df_merge['Reviews'].str.replace("&#039;",'\'')
df_merge

,Drug,Condition,Reviews,Rating,Website
0,guanfacine,adhd,"""my son is halfway through his fourth week of ...",8.0,drugs.com
1,guanfacine,adhd,"""the actavis generic version of this medicatio...",1.0,drugs.com
2,guanfacine,adhd,"""my son was just diagnosed adhd today. he's 5 ...",7.0,drugs.com
3,guanfacine,adhd,"""the first few days on 1 mg in the morning, he...",4.0,drugs.com
4,guanfacine,adhd,"""ours 8 year old son as done so much better wi...",10.0,drugs.com
...,...,...,...,...,...
337348,isoniazid,tuberculosis,i am having some of these side effects and wil...,2.0,webmd
337349,isoniazid,tuberculosis,"made me tired,achy,and was told not to take st...",2.0,webmd
337350,isoniazid,tuberculosis,it makes me feel like crap after i take it.\ni...,3.0,webmd
337351,isoniazid,tuberculosis,when i strated taking the medication i was fin...,2.0,webmd


In [27]:
def extract_review_emotion(df,column):
    new_df = df.copy()
    
    filepath = "NRC-Emotion-Lexicon/NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
    emolex_df = pd.read_csv(filepath,  names=["word", "emotion", "association"], sep='\t')
    emolex_words = emolex_df.pivot(index='word',columns='emotion',values='association').reset_index()
    
    emotions = emolex_words.columns.drop('word')
    emo_df = pd.DataFrame(0, index=df.index, columns=emotions)

    stemmer = SnowballStemmer("english")
    
    with tqdm(total=len(list(new_df.iterrows()))) as pbar:
        for i, row in new_df.iterrows():
            pbar.update(1)
            document = word_tokenize(new_df.loc[i][column])
            for word in document:
                word = stemmer.stem(word.lower())
                emo_score = emolex_words[emolex_words.word == word]
                if not emo_score.empty:
                    for emotion in list(emotions):
                        emo_df.at[i, emotion] += emo_score[emotion]

    new_df = pd.concat([new_df, emo_df], axis=1)

    return new_df

In [ ]:
emotion_df = extract_review_emotion(df_merge, 'Reviews')
emotion_df.to_csv('emotion_sentiment.csv',index=False)

In [4]:
read_emotions_df = pd.read_csv('emotion_sentiment.csv')
rev_samp = read_emotions_df.sample(n=5,random_state = 42)
rev_samp

,Drug,Condition,Reviews,Rating,Website,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
232363,adderall,adhd,add with mild ocd as well. the med is very eff...,8.0,webmd,1,0,1,1,0,3,1,1,0,1
188173,generess fe,birth control,after reading everyone's horrifying posts i wa...,9.0,webmd,1,6,0,1,1,1,4,0,3,3
14335,etonogestrel,birth control,"""i had the implanon after i had my son. i was ...",2.0,drugs.com,0,0,0,0,0,0,0,0,2,0
198901,benzonatate,cough,i have taken the meds for one day and feeling ...,10.0,webmd,0,0,1,0,0,1,0,0,0,0
92841,mucinex d,cough and nasal congestion,"""twice this year i have had a cold and used th...",8.0,drugs.com,1,2,0,1,0,2,3,0,0,3
